En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [20]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
from pandas import json_normalize

#file_path = "farmers-protest-tweets-2021-2-4.json"
file_path2 = "C:\\Users\\JuanJo\\Desktop\\LATAM\\Challenge\\farmers-protest-tweets-2021-2-4.json"



In [45]:
df0=pd.read_json(file_path2, lines=True)
df=df0.dropna(axis=1, how='all')

In [67]:
df_users = json_normalize(df['user'])

In [68]:
df_acotado2=df[['id','media']]
df_acotado3=df[['id','quotedTweet']]
df_acotado4=df[['id','mentionedUsers']]

In [82]:
#dato=df_acotado2.iloc[0].media

In [84]:
df_expandido = df_acotado2.explode('media')
df_media = json_normalize(df_expandido['media'])
df_media['id_prim'] = df_expandido['id'].values
df_media=df_media.dropna(axis=1, how='all')
df_media2 = df_media[df_media['type'].notnull()]

In [78]:
df_quotedTweet = json_normalize(df['quotedTweet'])
df_quotedTweet['id_prim'] = df_acotado3['id'].values
df_quotedTweet2 = df_quotedTweet[df_quotedTweet['id'].notnull()]
df_quotedTweet3=df_quotedTweet2.dropna(axis=1, how='all')


In [69]:
df_expandido = df_acotado4.explode('mentionedUsers')
df_mentionedUsers = json_normalize(df_expandido['mentionedUsers'])
df_mentionedUsers['id_prim'] = df_expandido['id'].values
df_mentionedUsers=df_mentionedUsers.dropna(axis=1, how='all')
df_mentionedUsers2 = df_mentionedUsers[df_mentionedUsers['id'].notnull()]